## Step 1 - Connect to Ethereum Node

In [68]:
# Step 1a - Start local geth node
# Run runGeth script    
# $ ./00_runGeth.sh
# 
# Or the following but remember to include the preloaded keyfiles for all actors 
# geth --dev --dev.period 1 --datadir ./testchain --rpc --rpccorsdomain '*' --rpcport 8646 --rpcapi "eth,net,web3,debug" --port 32323 --maxpeers 0 --targetgaslimit 994712388 console

In [69]:
# If no module named solc
#!pip install --upgrade pip
#!pip install py-solc

In [70]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import web3, solc, json
from web3.contract import ConciseContract
from web3 import Web3
from web3.auto.gethdev import w3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
# Pointer to Ethereum node (Can also be grenache or infura)
ipc_path = '../testchain/geth.ipc'
w3 = Web3(Web3.IPCProvider(ipc_path))

In [30]:
# If using a geth node (optional)
from web3.middleware import geth_poa_middleware
w3.middleware_stack.inject(geth_poa_middleware, layer=0)

#### Check connection 

In [37]:
# Check that our geth node is running
block = w3.eth.getBlock('latest')
block.number

954

In [38]:
block.hash

HexBytes('0x5ce092d9111643d90104d60377d16d133a9eb16113fd841e0befbe478fe37e09')

In [39]:
# Checks that test accounts have loaded correctly. 
w3.eth.accounts[2]

# >'0xa22AB8A9D641CE77e06D98b7D7065d324D3d6976'

'0xa22AB8A9D641CE77e06D98b7D7065d324D3d6976'

## Step 2 - Set Up Environment

In [72]:
# compile contract, check which version. ie solidity 0.5.3
# !pip install solc
!solc --version

solc, the solidity compiler commandline interface
Version: 0.5.3+commit.10d17f24.Darwin.appleclang


In [73]:
# load web3 python helper functions 
SCRIPTSDIR="scripts"
!cp $SCRIPTSDIR/* .
import functions

In [74]:
# File operations for solidity flattener
!chmod 755 ./scripts/solidityFlattener.pl
!mkdir flattened

mkdir: flattened: File exists


## Step 3 - Deploy Base Smart Contracts

In [43]:
# define
def sign_transaction(txn):
    # Flesh out the transaction for local signing
    next_nonce = w3.eth.getTransactionCount(str(w3.eth.accounts[0]))
    signable_transaction = dict(
      txn,
      nonce=next_nonce,
      gasPrice=w3.toWei(4, 'gwei'),
    )
    # Sign transaction
    encrypted_key = open('./testchain/keystore/UTC--2017-05-20T02-37-30.360937280Z--a00af22d07c87d96eeeb0ed583f8f6ac7812827e').read()
    private_key = w3.eth.account.decrypt(encrypted_key, '')
    signature_info = w3.eth.account.signTransaction(signable_transaction, private_key)
    # Broadcast transaction
    txn_hash = w3.eth.sendRawTransaction(signature_info.rawTransaction)
    # Wait for the transaction to be mined
    receipt = w3.eth.waitForTransactionReceipt(txn_hash)

    return receipt

In [44]:
## Warning, this is for testing purposes only, this is not a secure way to handle private key. Needs to be rewritten.

### Step 3a - Contract Deployment

#### Build Contract

In [67]:
WHITELISTSOL = "./flattened/WhiteList_flattened.sol"
!./scripts/solidityFlattener.pl --contractsdir=./contracts --mainsol=WhiteList.sol --outputsol={WHITELISTSOL} # --verbose

In [53]:
WhiteListOutput = !solc --optimize --combined-json abi,bin,interface {WHITELISTSOL}
i = 0
while (i < len(WhiteListOutput)):
    if WhiteListOutput[i][:10] == '{"contract':
        print(WhiteListOutput[i][:20])
        WhiteListOutputContract = WhiteListOutput[i]
    i = i + 1

WhiteListAbi = json.loads(WhiteListOutputContract)['contracts'][WHITELISTSOL + ':WhiteList']['abi'] 
WhiteListBin = json.loads(WhiteListOutputContract)['contracts'][WHITELISTSOL + ':WhiteList']['bin']

# Build the contract object in preparation for deployment


{"contracts":{"./fla


In [54]:
WhiteListOutput

['{"contracts":{"./flattened/WhiteList_flattened.sol:Controlled":{"abi":"[{\\"constant\\":true,\\"inputs\\":[],\\"name\\":\\"initialised\\",\\"outputs\\":[{\\"name\\":\\"\\",\\"type\\":\\"bool\\"}],\\"payable\\":false,\\"stateMutability\\":\\"view\\",\\"type\\":\\"function\\"},{\\"constant\\":true,\\"inputs\\":[],\\"name\\":\\"isControllable\\",\\"outputs\\":[{\\"name\\":\\"\\",\\"type\\":\\"bool\\"}],\\"payable\\":false,\\"stateMutability\\":\\"view\\",\\"type\\":\\"function\\"},{\\"constant\\":false,\\"inputs\\":[],\\"name\\":\\"renounceOwnership\\",\\"outputs\\":[],\\"payable\\":false,\\"stateMutability\\":\\"nonpayable\\",\\"type\\":\\"function\\"},{\\"constant\\":true,\\"inputs\\":[],\\"name\\":\\"owner\\",\\"outputs\\":[{\\"name\\":\\"\\",\\"type\\":\\"address\\"}],\\"payable\\":false,\\"stateMutability\\":\\"view\\",\\"type\\":\\"function\\"},{\\"constant\\":true,\\"inputs\\":[],\\"name\\":\\"isOwner\\",\\"outputs\\":[{\\"name\\":\\"\\",\\"type\\":\\"bool\\"}],\\"payable\\":fals

In [55]:
WhiteListAbi

'[{"constant":true,"inputs":[],"name":"initialised","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"whiteList","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"isControllable","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"accounts","type":"address[]"}],"name":"remove","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"renounceOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"isOwner","outputs":[{"name":"","type":"bool"

#### Deploy WhiteList

In [57]:
## Deploy WhiteList
whitelist_deployer = w3.eth.contract(abi=WhiteListAbi, bytecode=WhiteListBin)
init = whitelist_deployer.constructor()
txn = init.buildTransaction({'gas': 1000000})
receipt = sign_transaction(txn)
WhiteListAddress = receipt.contractAddress
WhiteListAddress

'0x5fe7CD00aA1aA3cCf94F459E3fB5C633C640aBB4'

In [58]:
# Build ConciseContract artifact
whitelist_contract = w3.eth.contract(
    address=WhiteListAddress,
    abi=WhiteListAbi,
)
whitelist_CContract = ConciseContract(whitelist_contract)
whitelist_CContract

## Step 4 - Interacting with Smart Contract 

In [59]:
print("WhiteListAddress:             " + WhiteListAddress + "    Deployed:" + str(whitelist_CContract.isOwner()) )

WhiteListAddress:             0x5fe7CD00aA1aA3cCf94F459E3fB5C633C640aBB4    Deployed:True


#### 4a - Add accounts to whitelist

In [60]:
# Whitelist 
insert_list = [w3.eth.accounts[2], w3.eth.accounts[3]]

# Submit txn
txn = whitelist_contract.functions.add(insert_list).buildTransaction({'gas': 320000})
receipt = sign_transaction(txn)
receipt.status

1

In [61]:
# Using the standard web3 contract call 
dst_name = whitelist_contract.functions.isInWhiteList(w3.eth.accounts[3]).call()
dst_name

True

In [62]:
# Using the ConciseContract call 
whitelist_CContract.isInWhiteList(w3.eth.accounts[3])

True

#### 4b - Checking Controller

In [63]:
whitelist_CContract.isControllable()

True

In [64]:
# Submit txn to add new controller "accounts[5]"
txn = whitelist_contract.functions.addController(w3.eth.accounts[3]).buildTransaction({'gas': 320000})
receipt = sign_transaction(txn)
receipt.status

1

## Misc Code Blocks

In [65]:
## Get more information from a transaction hash
w3.eth.getTransaction(receipt.transactionHash)

AttributeDict({'blockHash': HexBytes('0x806f4997c19c3ebdee69ff5908dd788b20132771e8aec71f73e7e6080d4b38e3'),
 'blockNumber': 1226,
 'from': '0xa00Af22D07c87d96EeeB0Ed583f8F6AC7812827E',
 'gas': 320000,
 'gasPrice': 4000000000,
 'hash': HexBytes('0x03cd8cbf6855adda91f72bc1c6d67cf17d2e4f957128300c990148474ee43b7b'),
 'input': '0xa7fc7a07000000000000000000000000a33a6c312d9ad0e0f2e95541beed0cc081621fd0',
 'nonce': 21,
 'r': HexBytes('0x181af65fb8ea46e86c768570e10756bdf8f15486888cc5981f979da80925fc4b'),
 's': HexBytes('0x42967e091c4fa45843592275d14197278a5efd32a0b76402b3b2db5261b3861b'),
 'to': '0x5fe7CD00aA1aA3cCf94F459E3fB5C633C640aBB4',
 'transactionIndex': 0,
 'v': 28,
 'value': 0})

In [66]:
## Get the contracts final memory from a transaction receipt
w3.providers[0].make_request("debug_traceTransaction", [str(ascii(receipt.transactionHash))[10:-2], {}])['result']['structLogs'][-1]['memory']

['000000000000000000000000a33a6c312d9ad0e0f2e95541beed0cc081621fd0',
 '0000000000000000000000000000000000000000000000000000000000000001',
 '0000000000000000000000000000000000000000000000000000000000000080',
 '0000000000000000000000000000000000000000000000000000000000000000',
 '000000000000000000000000a33a6c312d9ad0e0f2e95541beed0cc081621fd0']